<a href="https://colab.research.google.com/github/CorralesK/Lab04_EDA/blob/main/lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laboratorio 04 - EDA**

In [ ]:
# Cargando las librerias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## 1. Carga y comprensión inicial de los datos

In [ ]:
# Cargar el dataset


In [ ]:
# Verificar el formato


In [ ]:
# Explorar las primeras filas


In [ ]:
# Comprobar el tipo de cada


In [ ]:
# Observar la cantidad de filas y columnas


## 2. Limpiar y preparar los datos

In [ ]:
# Identificar y tratar valores nulos


In [ ]:
# Eliminar duplicados


In [ ]:
# Revisar valores atípicos


In [ ]:
# Convertir tipos de datos


In [ ]:
# Cambiar nombre de columnas si procede


In [ ]:
# Tratar con variables categóricas


## 3. Análisis univariado

In [ ]:
# Estadísticas descriptivas


In [ ]:
# Histograma y distribución de datos


In [ ]:
# Comprobación de asimetría y normalidad
